# Machine Learning Foundation

## Course 5, Part e: CNN DEMO

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

import matplotlib.pyplot as plt

In [2]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

## Each image is a 32 x 32 x 3 numpy array
print(x_train[444].shape)
print(y_train[444])
#plt.imshow(x_train[444]);

170508288/170498071 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(32, 32, 3)
[9]


In [3]:
from tensorflow.keras.utils import to_categorical

num_classes = 10

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# now instead of classes described by an integer between 0-9 we have a vector with a 1 in the (Pythonic) 9th position
print(y_train[444])

# As before, let's make everything float and scale
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [22]:
# Let's build model_1: 2-1, 5x5 architecture.

model_1 = Sequential()


## 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

## Another 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(64, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

## 2x2 max pooling reduces to 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.2))

## Flatten turns 3x3x32 into 288x1
model_1.add(Flatten())
model_1.add(Dense(256))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.4))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 16, 16, 32)        2432      
_________________________________________________________________
activation_76 (Activation)   (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 6, 6, 64)          51264     
_________________________________________________________________
activation_77 (Activation)   (None, 6, 6, 64)          0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 3, 3, 64)          0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 576)             

We still have 181K parameters, even though this is a "small" model.


In [23]:
batch_size = 32

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0005, decay=1e-6)

# Let's train the model using RMSprop
model_1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

#t = time.time()
model_1.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test, y_test),
              shuffle=True)

#print('elpsed time is', time.time()-t)

Epoch 1/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.6984 - accuracy: 0.3845 - val_loss: 1.4440 - val_accuracy: 0.4734
Epoch 2/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.3733 - accuracy: 0.5105 - val_loss: 1.2969 - val_accuracy: 0.5402
Epoch 3/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.2482 - accuracy: 0.5563 - val_loss: 1.1664 - val_accuracy: 0.5874
Epoch 4/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1702 - accuracy: 0.5868 - val_loss: 1.1208 - val_accuracy: 0.6162
Epoch 5/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1190 - accuracy: 0.6064 - val_loss: 1.0835 - val_accuracy: 0.6261
Epoch 6/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0840 - accuracy: 0.6228 - val_loss: 1.0560 - val_accuracy: 0.6335
Epoch 7/10
1563/1563 [==============================] - 9s 5ms/step - loss: 1.0545 - accuracy: 0.6339 - val_loss: 1.0586 - val_accuracy

### Exercise
Our previous model had the structure:

Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Final Classification

(with appropriate activation functions and dropouts)

1. Build a more complicated model with the following pattern:
- Conv -> Conv -> MaxPool -> Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Final Classification

- Use strides of 1 for all convolutional layers.

2. How many parameters does your model have?  How does that compare to the previous model?

3. Train it for 5 epochs.  What do you notice about the training time, loss and accuracy numbers (on both the training and validation sets)?

5. Try different structures and run times, and see how accurate your model can be.


In [10]:
# Let's build model_2: 2-1-2-1, 3x3 architecture

model_2 = Sequential()

model_2.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_2.add(Activation('relu'))
model_2.add(Conv2D(32, (3, 3)))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.2))

model_2.add(Conv2D(64, (3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv2D(64, (3, 3)))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.2))

model_2.add(Flatten())
model_2.add(Dense(512))
model_2.add(Activation('relu'))
model_2.add(Dropout(0.4))
model_2.add(Dense(num_classes))
model_2.add(Activation('softmax'))

model_2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_24 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_25 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 15, 15, 64)       

In [7]:
# initiate RMSprop optimizer
opt_2 = RMSprop(lr=0.0005)

# Let's train the model using RMSprop
model_2.compile(loss='categorical_crossentropy',
              optimizer=opt_2,
              metrics=['accuracy'])

model_2.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test, y_test),
              shuffle=True)

Epoch 1/10
1563/1563 [==============================] - 13s 7ms/step - loss: 1.5362 - accuracy: 0.4427 - val_loss: 1.1852 - val_accuracy: 0.5823
Epoch 2/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1203 - accuracy: 0.6056 - val_loss: 1.1283 - val_accuracy: 0.6074
Epoch 3/10
1563/1563 [==============================] - 12s 7ms/step - loss: 0.9453 - accuracy: 0.6717 - val_loss: 0.9269 - val_accuracy: 0.6889
Epoch 4/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8476 - accuracy: 0.7064 - val_loss: 0.8418 - val_accuracy: 0.7097
Epoch 5/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7899 - accuracy: 0.7277 - val_loss: 0.8641 - val_accuracy: 0.7113
Epoch 6/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7573 - accuracy: 0.7401 - val_loss: 0.7459 - val_accuracy: 0.7469
Epoch 7/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7476 - accuracy: 0.7480 - val_loss: 0.8305 - val_ac

In [18]:
# Let's build model_3: 2-1-2-1-1, 3x3 architecture, lower dropouts

model_3 = Sequential()

model_3.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_3.add(Activation('relu'))
model_3.add(Conv2D(32, (3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.2))

model_3.add(Conv2D(64, (3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(Conv2D(64, (3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.2))

model_3.add(Conv2D(128, (3, 3)))
model_3.add(Activation('relu'))
#model_3.add(MaxPooling2D(pool_size=(2, 2)))
#model_3.add(Dropout(0.2))

model_3.add(Flatten())
model_3.add(Dense(512))
model_3.add(Activation('relu'))
model_3.add(Dropout(0.4))
model_3.add(Dense(num_classes))
model_3.add(Activation('softmax'))

model_3.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_65 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_66 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 15, 15, 64)      

In [19]:
# initiate RMSprop optimizer
opt_3 = RMSprop(lr=0.0005)

# Let's train the model using RMSprop
model_3.compile(loss='categorical_crossentropy',
              optimizer=opt_3,
              metrics=['accuracy'])

model_3.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test, y_test),
              shuffle=True)

Epoch 1/10
1563/1563 [==============================] - 14s 8ms/step - loss: 1.6027 - accuracy: 0.4143 - val_loss: 1.4312 - val_accuracy: 0.4928
Epoch 2/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.1785 - accuracy: 0.5828 - val_loss: 0.9978 - val_accuracy: 0.6451
Epoch 3/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.9803 - accuracy: 0.6578 - val_loss: 0.9127 - val_accuracy: 0.6824
Epoch 4/10
1563/1563 [==============================] - 13s 8ms/step - loss: 0.8573 - accuracy: 0.7034 - val_loss: 0.8222 - val_accuracy: 0.7170
Epoch 5/10
1563/1563 [==============================] - 12s 7ms/step - loss: 0.7772 - accuracy: 0.7330 - val_loss: 0.7830 - val_accuracy: 0.7404
Epoch 6/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.7336 - accuracy: 0.7511 - val_loss: 0.7444 - val_accuracy: 0.7503
Epoch 7/10
1563/1563 [==============================] - 13s 8ms/step - loss: 0.6970 - accuracy: 0.7645 - val_loss: 0.8957 - val_ac

---
### Machine Learning Foundation (C) 2020 IBM Corporation